In [1]:
import cv2
import numpy as np
import pickle

In [1]:
import cv2
import threading
import time

def capture_frame(camera_index, results, lock):
    """
    ????? ???? ???? ??
    :param camera_index: ??? ??? ???
    :param results: ??? ??? ????
    :param lock: ??? ? ???? ?? Lock
    """
    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        print(f"Cannot open camera {camera_index}")
        return

    time.sleep(0.1) 

    ret, frame = cap.read()
    if ret:
        with lock:
            results[camera_index] = frame
    else:
        print(f"Failed to grab frame from camera {camera_index}")
    cap.release()

def capture_images_simultaneously(camera_indices):
    """
    ?? ????? ??? ???? ???? ??
    :param camera_indices: ??? ??? ??? ???
    :return: ??? ??? ???
    """
    threads = []
    results = {}
    lock = threading.Lock()

    # ? ???? ?? ????? ??
    for idx in camera_indices:
        thread = threading.Thread(target=capture_frame, args=(idx, results, lock))
    
        threads.append(thread)
        thread.start()

    # ?? ???? ??? ??? ??? ??
    for thread in threads:
        thread.join()

    # ??? ??? ??? ???? ???? ??
    return [results[idx] for idx in camera_indices if idx in results]

# ?? ??
camera_indices = ['/dev/video0', '/dev/video4', '/dev/video8', '/dev/video25']  # ??? ??? (?: /dev/video0, /dev/video1 ...)
images = capture_images_simultaneously(camera_indices)

if images:
    for i, img in enumerate(images):
        cv2.imshow(f"Camera {camera_indices[i]}", img)
    cv2.imwrite(f"camera_{i}.jpg", img)  
    cv2.waitKey(0)
    cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/pi/.local/lib/python3.11/site-packages/cv2/qt/plugins"


In [18]:
images

[array([[[ 50,  56,  39],
         [ 48,  53,  37],
         [ 38,  58,  19],
         ...,
         [ 18,  53,  36],
         [ 53,  94,  41],
         [184, 224, 172]],
 
        [[ 48,  55,  42],
         [ 47,  53,  41],
         [ 47,  57,  26],
         ...,
         [ 27,  48,  33],
         [ 38,  59,  26],
         [ 73,  94,  61]],
 
        [[ 45,  57,  42],
         [ 41,  53,  37],
         [ 40,  52,  30],
         ...,
         [ 34,  50,  29],
         [ 42,  50,  33],
         [ 54,  62,  45]],
 
        ...,
 
        [[241, 241, 241],
         [241, 241, 241],
         [241, 241, 241],
         ...,
         [ 34,  53,  31],
         [ 30,  58,  31],
         [ 29,  57,  29]],
 
        [[241, 241, 241],
         [241, 241, 241],
         [241, 241, 241],
         ...,
         [ 34,  52,  33],
         [ 39,  58,  35],
         [ 41,  60,  37]],
 
        [[241, 241, 241],
         [241, 241, 241],
         [241, 241, 241],
         ...,
         [ 34,  55,  35],
  

In [ ]:
def load_calibration_data(camera_indices):
    calibration_data = {}
    for idx in camera_indices:
        with open(f'calibration_data_camera{idx}.pkl', 'rb') as f:
            data = pickle.load(f)
            calibration_data[idx] = {
                'K': data['K'],
                'D': data['D'],
                'DIM': data['img_shape']  # (width, height)
            }
    return calibration_data

pkl_indices = [0,1,2,3]
calibration_data = load_calibration_data(pkl_indices)


In [3]:
import cv2

def save_images(images, output_folder="output"):
    """
    ??? ???? ??? ??? JPG ???? ??
    :param images: ???? ??? ???
    :param output_folder: ???? ??? ?? ??
    """
    import os

    # ?? ??
    os.makedirs(output_folder, exist_ok=True)

    # ? ???? ???? ??
    for i, img in enumerate(images):
        filename = os.path.join(output_folder, f"image_{i}.jpg")
        cv2.imwrite(filename, img)
        print(f"Saved: {filename}")

# ?? ??
# images: ? ???? numpy ??? ??? ???
# ?: [img1, img2, img3, img4]
save_images(images)


Saved: output/image_0.jpg
Saved: output/image_1.jpg
Saved: output/image_2.jpg
Saved: output/image_3.jpg
